In [2]:
pip install pydub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import subprocess

result = subprocess.run(
    ["ffmpeg", "-version"],
    capture_output=True,
    text=True
)

print(result.stdout)


ffmpeg version 2025-10-12-git-0bc54cddb1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --

In [2]:
choco install ffmpeg


SyntaxError: invalid syntax (2466941882.py, line 1)

In [4]:
import os
from pydub import AudioSegment
from openai import OpenAI

# -----------------------------
# Configuration
# -----------------------------

# Path to the original audio file
AUDIO_PATH = r"C:\Users\kupit\week 1\d4\lab1\ALDDEBWDBIUQND8A-M-b256-e8a94.mp4"

# Temporary trimmed audio file (first 60 seconds)
TRIMMED_AUDIO_PATH = r"C:\Users\kupit\week 1\d4\lab1\audio_first_60_seconds.wav"

# Output transcription file
OUTPUT_TEXT_PATH = r"C:\Users\kupit\week 1\d4\lab1\transcription_first_minute.txt"

# Initialize OpenAI client
client = OpenAI()

# -----------------------------
# Step 2: Verify audio file
# -----------------------------

if not os.path.exists(AUDIO_PATH):
    raise FileNotFoundError(f"Audio file not found: {AUDIO_PATH}")

file_size_mb = os.path.getsize(AUDIO_PATH) / (1024 * 1024)
file_extension = os.path.splitext(AUDIO_PATH)[1]

print("Audio file verification:")
print(f"- Path: {AUDIO_PATH}")
print(f"- File size: {file_size_mb:.2f} MB")
print(f"- File format: {file_extension}")
print("Audio file is accessible.\n")

# -----------------------------
# Step 2.1: Extract first 60 seconds
# -----------------------------

print("Extracting first 60 seconds of audio...")

audio = AudioSegment.from_file(AUDIO_PATH)
first_minute_audio = audio[:60 * 1000]  # 60 seconds in milliseconds

first_minute_audio.export(TRIMMED_AUDIO_PATH, format="wav")

print(f"Trimmed audio saved to: {TRIMMED_AUDIO_PATH}\n")

# -----------------------------
# Step 3: Transcribe trimmed audio
# -----------------------------

with open(TRIMMED_AUDIO_PATH, "rb") as audio_file:
    transcription = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
    )

transcription_text = transcription.text

# -----------------------------
# Step 3.1: Save transcription
# -----------------------------

with open(OUTPUT_TEXT_PATH, "w", encoding="utf-8") as text_file:
    text_file.write(transcription_text)

# -----------------------------
# Output results
# -----------------------------

print("Transcription completed successfully.")
print(f"Transcription saved to: {OUTPUT_TEXT_PATH}\n")

print("Transcription preview:")
print("---------------------")
print(transcription_text)


Audio file verification:
- Path: C:\Users\kupit\week 1\d4\lab1\ALDDEBWDBIUQND8A-M-b256-e8a94.mp4
- File size: 23.83 MB
- File format: .mp4
Audio file is accessible.

Extracting first 60 seconds of audio...
Trimmed audio saved to: C:\Users\kupit\week 1\d4\lab1\audio_first_60_seconds.wav

Transcription completed successfully.
Transcription saved to: C:\Users\kupit\week 1\d4\lab1\transcription_first_minute.txt

Transcription preview:
---------------------
The following was recorded from Dictionary of American Regional English Tape 1-2-3-2, Side 1. This is a recording of Mrs. Nettie made at McConnellsville, Morgan County, Ohio, on the 21st of February, 1968, by The etched tape was used to record Nathaniel Noble's patch number 1666, and the staff of the local public information agency reviewed it and refactored it with the text, in addition to inviting the audience parts to be cleaned. We'll now hear Mrs. Nettie Nettie The story of Arthur the Rat. Once upon a time there was a young rat who 

In [13]:
import os
import json
from pydub import AudioSegment
from openai import OpenAI

# -----------------------------
# Configuration
# -----------------------------
AUDIO_PATH = r"C:\Users\kupit\week 1\d4\lab1\ALDDEBWDBIUQND8A-M-b256-e8a94.mp4"
OUTPUT_DIR = r"C:\Users\kupit\week 1\d4\lab1"

client = OpenAI()

# -----------------------------
# Load original audio
# -----------------------------
audio = AudioSegment.from_file(AUDIO_PATH)

# Limit to first 3 minutes
audio = audio[:3 * 60 * 1000]

# -----------------------------
# Split into 60-second chunks
# -----------------------------
chunk_length_ms = 60 * 1000  # 60 seconds
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

print(f"Total chunks: {len(chunks)}")  # should be 3

# -----------------------------
# Shorter robust prompt
# -----------------------------
guided_prompt = (
    "Transcribe this recorded meeting as accurately as possible. "
    "Audio may be quiet, unclear, or noisy. "
    "Preserve names, acronyms, and technical terms. "
    "Include full sentences and punctuation."
)

# -----------------------------
# Helper function to transcribe chunks
# -----------------------------
def transcribe_chunk(chunk, prompt=None, skip_empty=True):
    temp_chunk_path = "temp_chunk.wav"
    chunk.export(temp_chunk_path, format="wav")

    if skip_empty and chunk.dBFS < -50:
        os.remove(temp_chunk_path)
        return "", 0, 0  # return empty text with start/end

    start_ms = chunk.start_time if hasattr(chunk, "start_time") else 0
    end_ms = start_ms + len(chunk)

    with open(temp_chunk_path, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            prompt=prompt,
            response_format="text"
        )
    os.remove(temp_chunk_path)
    return transcription.strip(), start_ms, end_ms

# -----------------------------
# Functions to save with timestamps
# -----------------------------
def save_txt(chunks_info, path):
    with open(path, "w", encoding="utf-8") as f:
        for idx, (text, start_ms, end_ms) in enumerate(chunks_info):
            f.write(f"--- Chunk {idx+1} ---\n")
            f.write(f"Start: {start_ms} ms, End: {end_ms} ms\n")
            f.write(f"{text}\n\n")

def save_json(chunks_info, path):
    data = [{"chunk": idx+1, "start_ms": start_ms, "end_ms": end_ms, "text": text} 
            for idx, (text, start_ms, end_ms) in enumerate(chunks_info)]
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def save_srt(chunks_info, path):
    def ms_to_srt_time(ms):
        h = ms // 3600000
        m = (ms % 3600000) // 60000
        s = (ms % 60000) // 1000
        ms_rem = ms % 1000
        return f"{h:02}:{m:02}:{s:02},{ms_rem:03}"

    with open(path, "w", encoding="utf-8") as f:
        for idx, (text, start_ms, end_ms) in enumerate(chunks_info):
            f.write(f"{idx+1}\n")
            f.write(f"{ms_to_srt_time(start_ms)} --> {ms_to_srt_time(end_ms)}\n")
            f.write(f"{text}\n\n")

# -----------------------------
# Transcription for a version
# -----------------------------
def transcribe_version(prompt, skip_empty, prefix):
    chunks_info = []
    for idx, chunk in enumerate(chunks):
        # add start_time attribute for SRT and timestamps
        chunk.start_time = idx * chunk_length_ms
        version_name = "With Prompt" if prompt else "No Prompt"
        print(f"[{version_name}] Transcribing chunk {idx+1}/{len(chunks)} ...")
        text, start_ms, end_ms = transcribe_chunk(chunk, prompt=prompt, skip_empty=skip_empty)
        chunks_info.append((text, start_ms, end_ms))

    # Save in all formats
    save_txt(chunks_info, os.path.join(OUTPUT_DIR, f"{prefix}.txt"))
    save_json(chunks_info, os.path.join(OUTPUT_DIR, f"{prefix}.json"))
    save_srt(chunks_info, os.path.join(OUTPUT_DIR, f"{prefix}.srt"))

# -----------------------------
# Run baseline (no prompt)
# -----------------------------
transcribe_version(prompt=None, skip_empty=True, prefix="transcription_no_prompt_3min")

# -----------------------------
# Run guided prompt version
# -----------------------------
transcribe_version(prompt=guided_prompt, skip_empty=False, prefix="transcription_with_prompt_3min")

print("All transcriptions completed: TXT, JSON, SRT for both versions with timestamps.")


Total chunks: 3
[No Prompt] Transcribing chunk 1/3 ...
[No Prompt] Transcribing chunk 2/3 ...
[No Prompt] Transcribing chunk 3/3 ...
[With Prompt] Transcribing chunk 1/3 ...
[With Prompt] Transcribing chunk 2/3 ...
[With Prompt] Transcribing chunk 3/3 ...
All transcriptions completed: TXT, JSON, SRT for both versions with timestamps.
